In [53]:
import string

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import spacy


%matplotlib inline

In [54]:
dataset_path = 'wsb.csv'
data = pd.read_csv(dataset_path)

data

/var/folders/hp/pdsypshj7kb4657qmr31mgj80000gn/T/ipykernel_11028/1200012153.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(dataset_path)


,Unnamed: 0,title,score,id,url,comms_num,created,body,timestamp
0,0,"It's not about the money, it's about sending a...",55,l6ulcx,https://v.redd.it/6j75regs72e61,6.0,1.611863e+09,NaN,2021-01-28 21:37:41
1,1,Math Professor Scott Steiner says the numbers ...,110,l6uibd,https://v.redd.it/ah50lyny62e61,23.0,1.611862e+09,NaN,2021-01-28 21:32:10
2,2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47.0,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35
3,3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,l6ugk6,https://sec.report/Document/0001193125-21-019848/,74.0,1.611862e+09,NaN,2021-01-28 21:28:57
4,4,"Not to distract from GME, just thought our AMC...",71,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,156.0,1.611862e+09,NaN,2021-01-28 21:26:56
...,...,...,...,...,...,...,...,...,...
418534,365347,Comment,1,jc8m70h,NaN,0.0,1.678845e+09,"Please tell me you're not serious lmao, \n\nlo...",2023-03-15 01:56:41
418535,365348,Comment,1,jc8m6oi,NaN,0.0,1.678845e+09,Wanted to weigh in as someone who is exception...,2023-03-15 01:56:38
418536,365349,Comment,2,jc8m6gl,NaN,0.0,1.678845e+09,You are already down $25k on your short position?,2023-03-15 01:56:35
418537,365350,Comment,1,jc8m67v,NaN,0.0,1.678845e+09,"I'm in the UK, my equivilant of SPY is VUAG, i...",2023-03-15 01:56:33


In [55]:
data.dropna(subset=['body'], inplace=True)

In [56]:
data['original_body'] = data['body']

In [57]:
data.head()

,Unnamed: 0,title,score,id,url,comms_num,created,body,timestamp,original_body
2,2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47.0,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35,The CEO of NASDAQ pushed to halt trading “to g...
6,6,SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,l6uf6d,https://www.reddit.com/r/wallstreetbets/commen...,53.0,1.611862e+09,Hedgefund whales are spreading disinfo saying ...,2021-01-28 21:26:27,Hedgefund whales are spreading disinfo saying ...
7,7,THIS IS THE MOMENT,405,l6ub9l,https://www.reddit.com/r/wallstreetbets/commen...,178.0,1.611862e+09,Life isn't fair. My mother always told me that...,2021-01-28 21:19:31,Life isn't fair. My mother always told me that...
10,10,"We need to keep this movement going, we all ca...",222,l6uao1,https://www.reddit.com/r/wallstreetbets/commen...,70.0,1.611862e+09,I believe right now is one of those rare oppo...,2021-01-28 21:18:25,I believe right now is one of those rare oppo...
12,12,"Once you're done with GME - $AG and $SLV, the ...",0,l6u9wu,https://www.reddit.com/r/wallstreetbets/commen...,16.0,1.611861e+09,You guys are champs. GME... who would have tho...,2021-01-28 21:17:10,You guys are champs. GME... who would have tho...


In [61]:
# stock dictionary filter

stock_tags = {
    'GME': ['gamestop', 'gme'],
    'AMC': ['amc'],
    'BB': ['blackberry', 'bb'],
    'TSLA': ['tesla', 'tsla', 'elon', 'musk'],
    'PLTR': ['palantir', 'pltr'],
    'MSFT': ['microsoft', 'msft'],
    'BBBY': ['bed bath and beyond', 'bbby'],
    'SPY': ['spy'],
    'AAPL': ['apple', 'aapl'],
}

In [62]:
# for each stock, create a seperate df
stock_dfs = {}
for stock, tags in stock_tags.items():
    stock_dfs[stock] = data[data['body'].str.contains('|'.join(tags), case=False)]
    print(stock, stock_dfs[stock].shape)

GME (11982, 10)
AMC (3965, 10)
BB (10756, 10)
TSLA (12214, 10)
PLTR (793, 10)
MSFT (1133, 10)
BBBY (3783, 10)
SPY (6124, 10)


In [60]:
nlp = spacy.load('en_core_web_sm')

In [46]:
import re

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_stop_words(text):
    doc = nlp(text)
    return " ".join([token.text for token in doc if not token.is_stop])

def lemmatize_words(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

remove_spaces = lambda x : re.sub('\\s+', ' ', x)

# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

remove_double_quotes = lambda x : x.replace('"', '')
remove_single_quotes = lambda x : x.replace('\'', '')
trim = lambda x : x.strip()

other_chars = ['*', '#', '&x200B', '[', ']', '; ',' ;' "&nbsp", "“","“","”", "x200b"]
def remove_other_chars(x: str):
    for char in other_chars:
        x = x.replace(char, '')
    
    return x

def lower_case_text(text):
    return text.lower()


In [47]:

funcs = [
    remove_urls, 
    remove_punctuation,
    remove_stop_words, 
    remove_emoji, 
    remove_double_quotes, 
    remove_single_quotes,
    lower_case_text,
    remove_other_chars,
    lemmatize_words,
    remove_spaces,
    trim]

for fun in funcs:
    data['body'] = data['body'].apply(fun)

In [48]:

# reset indexes (again)
data.reset_index(inplace=True)
data.drop(['index'], axis=1, inplace=True)

#drop empty body
data.dropna(subset=['body'], inplace=True)

data

,Unnamed: 0,title,score,id,url,comms_num,created,body,timestamp,original_body
0,2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47.0,1.611862e+09,,2021-01-28 21:30:35,The CEO of NASDAQ pushed to halt trading “to g...
1,6,SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,l6uf6d,https://www.reddit.com/r/wallstreetbets/commen...,53.0,1.611862e+09,,2021-01-28 21:26:27,Hedgefund whales are spreading disinfo saying ...
2,7,THIS IS THE MOMENT,405,l6ub9l,https://www.reddit.com/r/wallstreetbets/commen...,178.0,1.611862e+09,,2021-01-28 21:19:31,Life isn't fair. My mother always told me that...
3,10,"We need to keep this movement going, we all ca...",222,l6uao1,https://www.reddit.com/r/wallstreetbets/commen...,70.0,1.611862e+09,,2021-01-28 21:18:25,I believe right now is one of those rare oppo...
4,12,"Once you're done with GME - $AG and $SLV, the ...",0,l6u9wu,https://www.reddit.com/r/wallstreetbets/commen...,16.0,1.611861e+09,,2021-01-28 21:17:10,You guys are champs. GME... who would have tho...
...,...,...,...,...,...,...,...,...,...,...
95,176,Chamath Palihapitiya for President,10675,l6kqyk,https://www.reddit.com/r/wallstreetbets/commen...,587.0,1.611827e+09,,2021-01-28 11:46:30,"This is simple retards, he already publicly hi..."
96,177,We. Are. Strong. 🚀🚀,3238,l6kicj,https://www.reddit.com/r/wallstreetbets/commen...,198.0,1.611827e+09,,2021-01-28 11:35:13,This community has gone through many things to...
97,178,Thank you moderators,2076,l6kgq9,https://www.reddit.com/r/wallstreetbets/commen...,90.0,1.611826e+09,,2021-01-28 11:33:12,I don’t know what the future of this community...
98,179,Explanation for noobs who don't quite understa...,13228,l6kgik,https://www.reddit.com/r/wallstreetbets/commen...,1331.0,1.611826e+09,,2021-01-28 11:32:55,Mods: does DD tag apply? If not change it.\n\n...
